In [4]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random

In [23]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [40]:
req1 = requests.get("https://www.eventscribe.net/2022/nacfcprogram/agenda.asp?pfp=Browse%20by%20Day")
soup = BeautifulSoup(req1.content, 'html.parser')

In [42]:
for i in soup.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
    code = i.get('data-presid')
    print(code)

1148840
1148843
1087159
1087156
1087154
1087149
1148858
1087153
1148868
1148880
1148881
1148884
1148885
1148883
1148852
1148854
1148853
1148851
1148850
1148849
1148848
1087152
1087150
1087157
1087158
1087155
1087151
1087148
1099896
1148846
1148847
1148844
1148842
1148841
1148866
1148863
1148864
1154502
1148862
1099817
1099886
1150071
1148861
1087146
1087174
1087177
1087172
1087171
1087175
1087176
1087178
1087173
1099891
1087147
1087167
1087170
1087169
1087168
1087165
1087164
1087163
1087160
1087161
1087162
1087166
1087179
1150070
1088641
1088640
1089163
1126860
1126859
1126858
1126857
1126856
1126855
1126854
1126853
1088639
1127764
1127769
1127768
1127767
1127766
1127770
1127771
1088638
1148859
1148857
1088637
1150080
1148856
1148855
1088635
1088856
1088636
1147020
1147019
1147018
1147017
1147016
1147108
1147107
1147106
1088855
1128125
1128124
1128123
1128122
1128121
1128119
1128118
1150079
1099893
1088854
1088633
1148875
1088853
1088852
1088851
1088697
1088696
1088695
1088694
1088693


In [25]:
c = 0
se_no = 1

for ses in soup.find_all(class_='list-group-item bucket'):
    
    sess_id = f'S{se_no}'
    session_title = ses.find(style='color:#5A5A5A').text.strip()
    print(session_title)
    sess_ke_presentation = ses.next_sibling
    
    for i in sess_ke_presentation.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
        code = i.get('data-presid')
        icon = ''
        if i.find_all('img'): # check if there is any icon
            print('yesss iconn......')
            for i in i.find_all('img'):
                icon += i.get('title') + '; '
                
        session_url = f'https://gestannual2023.eventscribe.net/fsPopup.asp?PresentationID={code}&mode=presInfo'
        
        req = requests.get(session_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        art_title = soup.find('h1').text.strip()
        session_date = soup.find(class_='pull-left pres-tidbit tipsytip').text.strip()
        ses_time_loc = soup.find_all(class_='pull-left pres-tidbit')
        session_time = ses_time_loc[0].text.strip()

        if len(ses_time_loc) > 1:
            session_loc = ses_time_loc[1].text.strip()
        else:
            print('no location....')
            session_loc = ''

        sess_text = ''
        session_author = ''
        affiliation = ''
        sess_type = ''
        category = ''
        cat = soup.find_all(class_='trackname innertracks')
        if len(cat) < 1:
            cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')

        try:
            sess_type = soup.find(style='color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8;').text.strip()
        except:
            try:
                sess_type = soup.find(style='margin-top:10px;').text.strip()
            except:
                sess_type = ''

        for i in cat:
            category += i.text.strip() + '; '

        auth = soup.find(class_='speakers-wrap')

        for s_i in auth.find_all(class_='row'): # author affiliation
            aut = s_i.find(class_='col-xs-12 col-md-6').a.text
            aff = s_i.find(class_="text-muted prof-text")
            aff = re.split('<br.*?>', str(aff))
            aff = ' '.join(aff)
            if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
                aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
            else:
                aff = ' '
            session_author += aut.strip() + '; '
            affiliation += aff + '; '

        try: # abs_text with giving spaces
            sess_text = soup.find(class_='PresentationAbstractText mar-top')
            sess_text = str(sess_text)
            sess_text = re.sub('<.*?>', ' ', sess_text)
        except:
            sess_text = ''
        if sess_text == None:
            sess_text = ''

        if art_title == sess_type:
            sess_type = ' '
            print('check if.......')

        category = re.sub(sess_type+'; ', '', category)
        print(f'art title :- {art_title}')
        print(f'session type :- {sess_type}')
        print(f'category :- {category}')
        print(f'session date :- {session_date}')
        print(f'session time :- {session_time}')
        print(f'session location :- {session_loc}')
        print(f'session author :- {session_author}')
        print(f'session aff :- {affiliation}')
        print(f'session text :- {sess_text}')

        print('======================='*5)
        
        if session_title.strip() == art_title:
            dic['news_type'].append('Session')
        else:
            dic['news_type'].append('Abstract')
            
        dic['session_id'].append(sess_id)
        dic['source_id'].append('')
        dic['manual_id'].append('')
        dic['article_title'].append(art_title)
        dic['url'].append(session_url)
        dic['authors'].append(session_author)
        dic['author_affiliation'].append(affiliation)
        dic['abstract_text'].append(sess_text)
        dic['date'].append(session_date)
        dic['start_time'].append(session_time)
        dic['end_time'].append('')
        dic['location'].append(session_loc)
        dic['session_title'].append(session_title)
        dic['session_type'].append(sess_type)
        dic['category'].append(category)
        dic['sub_category'].append('')
        dic['disclosure'].append(icon)
    
    se_no += 1
    c += 1

Session I: Masterclass Coils - Lecture + Demo
check if.......
art title :- Session I: Masterclass Coils - Lecture + Demo
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 8:00 AM – 9:30 AM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Jafar Golzarian, MD; 
session aff :- Professor of Radiology and Surgery Division Head, Vascular and Interventional Radiology University of Minnesota University of Minnesota Plymouth, Minnesota, United States</br>; 
session text :- None
check if.......
art title :- Masterclass Coil Demonstration
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 8:00 AM – 8:40 AM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Marc Sapoval, MD,PhD; Gary Siskin, Medical Director; 
session aff :- Université de Paris / Hopital Européen Georges Pompidou Université Paris Cité Paris, Ile-de-France, France; Albany Medical Center 

check if.......
art title :- Hands-on PAE
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 1:00 PM – 2:00 PM East Coast USA Time
session location :- Location: Empire West Ballroom
session author :- Shivank Bhatia, MD; Tiago Bilhim, MD, PhD, EBIR-ES, FCIRSE, FSIR; 
session aff :- University of Miami Miller School of Medicine University of Miami Miami, Florida, United States; Centro Hepato-Bílio-Pancreático e da Transplantação, Hospital Curry Cabral, CHULC Lisbon, Portugal; 
session text :- None
check if.......
art title :- Station 1 - Patient Work Up Pre-PAE
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 1:00 PM – 2:00 PM East Coast USA Time
session location :- Location: Empire West Ballroom
session author :- Mark Little, FRCR; Nassir Rostambeigi, MD MPH; 
session aff :- Interventional Radiology Consultant Royal Berkshire Hospital NHSFT University Department of Radiology, Royal Berkshire NHS University Foundatio

check if.......
art title :- Fair Embo: Where do we stand?
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 3:30 PM – 3:40 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Vincent Vidal, MD-PhD; 
session aff :- MD, PhD APHM Aix Marseille University Paris, Provence-Alpes-Cote d'Azur, France; 
session text :- None
check if.......
art title :- Panel Discussion and Q&A
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 3:40 PM – 4:00 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Yasuaki Arai, MD, FSIR, FCIRSE; Man-Deuk Kim, MD,PhD,FSIR; Karim A. Abd El Tawab, Dr. Karim Abd El Tawab (he/him/his); 
session aff :- National Cancer Center Hospital Jikei University School of Medicine, Saitama Medical University Tokyo, Tokyo, Japan; Professor, MD, PhD, FSIR Severance Hospital - Yonsei University Severance Hospital, Yonsei University Seoul, 

check if.......
art title :- Controversies
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 4:30 PM – 4:35 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Samdeep Mouli; 
session aff :- Northwestern University Feinberg School of Medicine Chicago, Illinois, United States; 
session text :- None
check if.......
art title :- How to deal with non-target vessels
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 4:35 PM – 4:40 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Tim McClure; 
session aff :-  ; 
session text :- None
check if.......
art title :- The embolics – how to and evidence
session type :-  
category :- 
session date :- Thursday, May 18, 2023
session time :- 4:40 PM – 4:45 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Karim A. Abd El Tawab, Dr. Karim Abd El Tawab (he/him/his);

check if.......
art title :- Session VII: Masterclass IO Vascular - Master your way out of Mystery: Emulate the Master
session type :-  
category :- 
session date :- Friday, May 19, 2023
session time :- 8:00 AM – 9:30 AM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Kamran Ahrar, MD, MBA; Daniel Brown, Professor and Vice-Chair (he/him/his); Robert Lewandowski, MD, FSIR; Marnix Lam, MD, PhD (he/him/his); Yasuaki Arai, MD, FSIR, FCIRSE; Sanjeeva Kalva, MD; 
session aff :- MD Anderson Cancer Center MD Anderson Cancer Center Houston, Texas, United States; Vanderbilt University Medical Center Vanderbilt University Medical Center Nashville, Tennessee, United States; Feinberg School of Medicine at Northwestern University Northwestern University Chicago, Illinois, United States; University Medical Center Utrech University Medical Center Utrecht Zeist, Utrecht, Netherlands; National Cancer Center Hospital Jikei University School of Medicine, Saitama Med

...

check if.......
art title :- Panel for discussing with cases/focused questions
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 1:40 PM – 1:50 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Miyuki Sone, MD; Raj Narayanan, MD; Roberto Iezzi, MD; Ronald S. Arellano, MD (he/him/his); 
session aff :- National Cancer Center Hospital Tokyo, Japan; FLORIDA INTERNATIONAL UNIVERSITY Miami, Florida, United States; Catholic University of the Sacred Heart Catholic University - Rome Rome, Lazio, Italy; Harvard Medical School Boston, Massachusetts, United States; 
session text :- None
check if.......
art title :- Panel for discussing with cases/focused questions
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 2:10 PM – 2:30 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Miyuki Sone, MD; Raj Narayanan, MD; Roberto Iezzi, MD; Ronald S. Arell

check if.......
art title :- Session XIX: Portal Interventions
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 4:00 PM – 5:30 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- John Andrew Kaufman, MD, MS; Wael Saad, MD; 
session aff :- Dotter Interventional Institute / Oregon Health &amp; Science University OHSU Lake Oswego, Oregon, United States; National Institute of Health (NIH) University of Utah Salt Lake City, Utah, United States; 
session text :- None
check if.......
art title :- Racial disparities in portal vein interventions.
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 4:00 PM – 4:10 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Premal Trivedi, MD MSc; 
session aff :- University of Colorado Aurora, Colorado, United States; 
session text :- None
check if.......
art title :- RTO: Which option for which patients?
ses

check if.......
art title :- Case discussion of complex renal embolization: trauma and iatrogenic injury
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 4:40 PM – 4:50 PM East Coast USA Time
session location :- Location: Empire East Ballroom
session author :- Masato Yamaguchi, MD; 
session aff :- Kobe University Hospital Kobe, Hyogo, Japan; 
session text :- None
check if.......
art title :- Case discussion of complex renal embolization: benign and malignant masses
session type :-  
category :- 
session date :- Saturday, May 20, 2023
session time :- 4:50 PM – 5:00 PM East Coast USA Time
session location :- Location: Empire East Ballroom
session author :- John Fritz Angle, MD; 
session aff :- University of Virginia University of Virginia CHARLOTTESVILLE, Virginia, United States; 
session text :- None
check if.......
art title :- New technologies: radial access, navigation, and Angio-CT
session type :-  
category :- 
session date :- Saturday, May 20, 

check if.......
art title :- How to maximize ultrasound guidance during sclerotherapy/embolization of high and low flow vascular malformations
session type :-  
category :- 
session date :- Sunday, May 21, 2023
session time :- 8:40 AM – 8:50 AM East Coast USA Time
session location :- Location: Empire East Ballroom
session author :- Mohammad Badran, MBBS; 
session aff :- King Faisal Specialist Hospital and Research Centre Riyadh, Saudi Arabia; 
session text :- None
check if.......
art title :- Vascular malformations when and how I will use drug therapy in combination or not with interventional therapies
session type :-  
category :- 
session date :- Sunday, May 21, 2023
session time :- 8:50 AM – 9:00 AM East Coast USA Time
session location :- Location: Empire East Ballroom
session author :- SHELLIE Josephs, MD (she/her/hers); 
session aff :- Stanford Lucile Packard Children's Hospital Stanford University Portola Valley, California, United States; 
session text :- None
check if.......
ar

check if.......
art title :- Working closely with infertility clinics can markedly enhance your interventional practice
session type :-  
category :- 
session date :- Sunday, May 21, 2023
session time :- 11:30 AM – 11:40 AM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Lindsay Machan, MD, FSIR; 
session aff :- Vancouver Hospital University of British Columbia Vancouver, British Columbia, Canada; 
session text :- None
check if.......
art title :- Case Based and Panel  Discussion
session type :-  
category :- 
session date :- Sunday, May 21, 2023
session time :- 11:40 AM – 12:00 PM East Coast USA Time
session location :- Location: Metro East Ballroom
session author :- Tim McClure; Jose Urbano, PhD, FCIRSE; Ahmed K. Abdel Aal, MD, PhD; Theresa M. Caridi, MD, FSIR; Kirema Garcia Reyes, MD; Gloria Salazar, MD, FSIR (she/her/hers); Lindsay Machan, MD, FSIR; 
session aff :-  ; Ramon y Cajal University Hospital Majadahonda, Madrid, Spain; University of

In [26]:
df = pd.DataFrame(dic)
df

,source_id,manual_id,article_title,url,authors,author_affiliation,abstract_text,date,start_time,end_time,location,session_id,news_type,session_title,session_type,category,sub_category,disclosure
0,,,Session I: Masterclass Coils - Lecture + Demo,https://gestannual2023.eventscribe.net/fsPopup...,"Jafar Golzarian, MD;",Professor of Radiology and Surgery Division He...,None,"Thursday, May 18, 2023",8:00 AM – 9:30 AM East Coast USA Time,,Location: Metro East Ballroom,S1,Session,Session I: Masterclass Coils - Lecture + Demo,,,,
1,,,Masterclass Coil Demonstration,https://gestannual2023.eventscribe.net/fsPopup...,"Marc Sapoval, MD,PhD; Gary Siskin, Medical Dir...",Université de Paris / Hopital Européen Georges...,None,"Thursday, May 18, 2023",8:00 AM – 8:40 AM East Coast USA Time,,Location: Metro East Ballroom,S1,Abstract,Session I: Masterclass Coils - Lecture + Demo,,,,
2,,,Rapid Fire Panel Discussion,https://gestannual2023.eventscribe.net/fsPopup...,"Dania Daye, MD, PhD; Parag J. Patel, MD MS (he...","Harvard Medical School Medford, Massachusetts,...",None,"Thursday, May 18, 2023",8:40 AM – 9:00 AM East Coast USA Time,,Location: Metro East Ballroom,S1,Abstract,Session I: Masterclass Coils - Lecture + Demo,,,,
3,,,Panel Video Discussions,https://gestannual2023.eventscribe.net/fsPopup...,"Dania Daye, MD, PhD; Parag J. Patel, MD MS (he...","Harvard Medical School Medford, Massachusetts,...",None,"Thursday, May 18, 2023",9:00 AM – 9:30 AM East Coast USA Time,,Location: Metro East Ballroom,S1,Abstract,Session I: Masterclass Coils - Lecture + Demo,,,,
4,,,Session II: Vanguard & Honarary Lecture,https://gestannual2023.eventscribe.net/fsPopup...,"Jafar Golzarian, MD; Marc Sapoval, MD,PhD;",Professor of Radiology and Surgery Division He...,None,"Thursday, May 18, 2023",10:00 AM – 11:30 AM East Coast USA Time,,Location: Metro East Ballroom,S2,Session,Session II: Vanguard & Honarary Lecture,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,,,Liver dominant mNET: When to advocate for embo...,https://gestannual2023.eventscribe.net/fsPopup...,"Sarah White, MD, MS, FSIR, FCIRSE;","Medical College of Wisconsin MCW Milwaukee, Wi...",None,"Sunday, May 21, 2023",11:10 AM – 11:20 AM East Coast USA Time,,Location: Empire East Ballroom,S30,Abstract,Session XXIV: Liver Mets,,,,
199,,,"FLR Hypertrophy: PVE, LVD, Y90",https://gestannual2023.eventscribe.net/fsPopup...,"Boris Guiu, MD, PhD;",St-Eloi University Hospital of Montpellier Mon...,None,"Sunday, May 21, 2023",11:20 AM – 11:30 AM East Coast USA Time,,Location: Empire East Ballroom,S30,Abstract,Session XXIV: Liver Mets,,,,
200,,,Panel Discussion (Questions TBD),https://gestannual2023.eventscribe.net/fsPopup...,"Sarah White, MD, MS, FSIR, FCIRSE; Boris Guiu,...","Medical College of Wisconsin MCW Milwaukee, Wi...",None,"Sunday, May 21, 2023",11:30 AM – 11:40 AM East Coast USA Time,,Location: Empire East Ballroom,S30,Abstract,Session XXIV: Liver Mets,,,,
201,,,What other cancers should we be targeting?,https://gestannual2023.eventscribe.net/fsPopup...,,,None,"Sunday, May 21, 2023",11:40 AM – 11:50 AM East Coast USA Time,,Location: Empire East Ballroom,S30,Abstract,Session XXIV: Liver Mets,,,,


In [28]:
df.to_excel('check_event.xlsx', index=False)